In [13]:
import mysql.connector
import pandas as pd

ojk = mysql.connector.connect(
    host = "database-1.cnu82kme6p4d.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = "eda",
    password = "ojk0707",
    database = "ojk"
)
cursor = ojk.cursor()
query = "SELECT * FROM cafe where score >3.0"
cafe = pd.read_sql(query, ojk)
cafe.head()

/tmp/ipykernel_13929/2553063093.py:13: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,station,score,review,address,latitude,longitude
0,(구 이문커피) 2MOON 이문 카페펍,동대문역,4.7,44,서울 종로구 창신1길 12 창신아지트 3층 이문카페펍(구 이문커피),37.572216,127.011411
1,#52LAB,회현역,4.5,44,서울 중구 퇴계로14길 19,37.559687,126.983781
2,0122,노량진역,3.9,24,서울 동작구 노량진로 162,37.513386,126.943922
3,10G,여의나루역,3.9,80,서울 영등포구 국제금융로 10 1층,37.524978,126.925298
4,10꼬르소꼬모서울카페,압구정로데오역,3.2,102,서울 강남구 압구정로 416 더트리니티빌딩 지하1층,37.526916,127.042419


In [14]:
import numpy as np

cafe['value'] = cafe['score'] * np.log(cafe['review']+1)
cafe.head()

,name,station,score,review,address,latitude,longitude,value
0,(구 이문커피) 2MOON 이문 카페펍,동대문역,4.7,44,서울 종로구 창신1길 12 창신아지트 3층 이문카페펍(구 이문커피),37.572216,127.011411,17.891314
1,#52LAB,회현역,4.5,44,서울 중구 퇴계로14길 19,37.559687,126.983781,17.129981
2,0122,노량진역,3.9,24,서울 동작구 노량진로 162,37.513386,126.943922,12.553616
3,10G,여의나루역,3.9,80,서울 영등포구 국제금융로 10 1층,37.524978,126.925298,17.138352
4,10꼬르소꼬모서울카페,압구정로데오역,3.2,102,서울 강남구 압구정로 416 더트리니티빌딩 지하1층,37.526916,127.042419,14.831133


In [15]:
import plotly.express as px

fig = px.histogram(cafe,x='value',title='평가지수별 가게 분포',color_discrete_sequence=['green'])
fig.update_layout(
    xaxis_title='평가지수 (평점 × log(리뷰수 + 1))',
    yaxis_title='카페 수',
    
)
fig.show()

In [16]:
avg_value = cafe['value'].mean()
print("전체 카페의 평균 평가지수:", avg_value)



전체 카페의 평균 평가지수: 14.303963193685409


In [17]:
import plotly.express as px

# 평균 평가지수 계산
avg_value = cafe['value'].mean()
print("전체 카페의 평균 평가지수:", avg_value)

# 히스토그램 생성
fig = px.histogram(cafe, x='value', title='평가지수별 가게 분포', color_discrete_sequence=['green'])

# 축 라벨 수정
fig.update_layout(
    xaxis_title='평가지수 (평점 × log(리뷰수 + 1))',
    yaxis_title='카페 수'
)

# y축 최대값 계산
y_max = cafe['value'].value_counts().max()

# 평균 평가지수선 추가
fig.add_shape(
    type='line',
    x0=avg_value,
    x1=avg_value,
    y0=0,
    y1=y_max,
    line=dict(color='black', dash='dash')
)

# 선에 라벨 추가
fig.add_annotation(
    x=avg_value,
    y=y_max,
    text=f"평균: {avg_value:.2f}",
    showarrow=False,
    yshift=10,
    font=dict(color='black')
)

fig.show()


전체 카페의 평균 평가지수: 14.303963193685409


In [80]:
4.5*np.log(15)

np.float64(12.186225904959946)

In [81]:
2.8*np.log(300)

np.float64(15.970590929037362)

In [18]:
# 역별 평균 value 계산
station_avg = cafe.groupby('station', as_index=False)['value'].mean()
station_avg = station_avg.sort_values(by='value', ascending=False)
# 막대그래프 시각화
fig = px.bar(
    station_avg,
    x='station',
    y='value',
    title='지하철역별 카페 평가지수 평균',
    labels={'station': '지하철역', 'value': '평균 평가지수 (평점 × log(리뷰수 + 1))'},
    color_discrete_sequence=['green']
)

fig.show()

In [19]:
# 상위 10개만 추출
top10 = station_avg.head(10)

# 그래프 그리기
fig = px.bar(
    top10,
    x='station',
    y='value',
    title='평균 평가지수 기준 상위 10개 지하철역',
    color_discrete_sequence=['red']

)
fig.update_layout(
    width=800,
    height=500,
    bargap=0.3,  # 바 간격 (막대 폭 줄이기 효과)
    title='평균 평가지수 기준 상위 10개 지하철역',
    xaxis_title='지하철역',
    yaxis_title='평균 평가지수'
)
fig.show()

In [20]:
# value 오름차순 정렬 (작은 값이 먼저 오게)
station_avg_sorted = station_avg.sort_values(by='value', ascending=True)

# 하위 10개만 추출
bottom10 = station_avg_sorted.head(10)
print("전체 카페의 평균 평가지수:", avg_value)

# 그래프 그리기
fig = px.bar(
    bottom10,
    x='station',
    y='value',
    title='평균 평가지수 기준 하위 10개 지하철역',
    color_discrete_sequence=['blue']
)

fig.update_layout(
    width=800,
    height=500,
    bargap=0.3,
    xaxis_title='지하철역',
    yaxis_title='평균 평가지수'
)

fig.show()

전체 카페의 평균 평가지수: 14.303963193685409
